In [14]:
import requests
import pandas as pd

HEADERS = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36'}

def print_error(self, error):
    print('[!] '+ str(error.response.status_code))
    print('[!]  '+ error.response.text)

class Server:
    """
    A remote SNOMED server.
    """

    def __init__(self, uri):
        self.server_uri = uri
        self.branch_uri = None
        # future: allow branch to be passed as a parameter and automatically filled if known
        # if no branch specified, print warning message + advice on getting branches

    def get_response_return_object(self, response, return_type):
        if return_type=='json':
            return response.json()
        elif return_type=='df':
            return pd.DataFrame( response.json()['items'] )
        else:
            print("Error: return_type should be either 'json' (for raw JSON) or 'df' (for pandas DataFrame)")


    def get_snomed_editions(self, return_type='df'):
        """
        Get the SNOMED editions available at the server.
        """
        try:
            r = requests.get(self.server_uri+'codesystems', headers=HEADERS)
            r.raise_for_status()

        except requests.exceptions.HTTPError as e:
            print_error(e)

        if return_type=='json':
            return r.json()
        elif return_type=='df':
            return pd.DataFrame( r.json()['items'] )
        else:
            print("Error: return_type should be either 'json' (for raw JSON) or 'df' (for pandas DataFrame)")

    def get_snomed_versions(self, edition, return_type='df'):
        """
        Get available SNOMED versions for a given edition
        """
        try:
            r = requests.get(self.server_uri+'codesystems/'+edition+'/versions', headers=HEADERS)
            r.raise_for_status()

        except requests.exceptions.HTTPError as e:
            print_error(e)

        return self.get_response_return_object(r, return_type)
    
    def set_branch(self, branch):
        self.branch_uri = self.server_uri + branch

        
# parses results as Pandas dataframes


In [15]:
snomed_server = Server('https://browser.ihtsdotools.org/snowstorm/snomed-ct/')
editions = snomed_server.get_snomed_editions()
editions.head()

,name,shortName,branchPath,dailyBuildAvailable,latestDailyBuild,countryCode,latestVersion,languages,modules,userRoles,owner,dependantVersionEffectiveTime,defaultLanguageReferenceSets,maintainerType,defaultLanguageCode
0,International Edition,SNOMEDCT,MAIN,True,2024-09-08-001550,,"{'shortName': 'SNOMEDCT', 'importDate': '2024-...",{'en': 'English'},"[{'conceptId': '900000000000012004', 'active':...",[],NaN,NaN,NaN,NaN,NaN
1,Argentinian Edition,SNOMEDCT-AR,MAIN/SNOMEDCT-ES/SNOMEDCT-AR,False,NaN,ar,"{'shortName': 'SNOMEDCT-AR', 'importDate': '20...","{'es': 'Spanish', 'en': 'English'}","[{'conceptId': '900000000000012004', 'active':...",[],Argentina Ministry of Health,20240331.0,[450828004],NaN,NaN
2,Austrian Edition,SNOMEDCT-AT,MAIN/SNOMEDCT-AT,True,2024-09-08-033132,at,"{'shortName': 'SNOMEDCT-AT', 'importDate': '20...","{'de': 'German', 'en': 'English'}","[{'conceptId': '900000000000012004', 'active':...",[],NaN,20240701.0,"[21000234103, 900000000000509007]",Managed Service,NaN
3,Australian Edition,SNOMEDCT-AU,MAIN/SNOMEDCT-AU,False,NaN,au,"{'shortName': 'SNOMEDCT-AU', 'importDate': '20...",{'en': 'English'},"[{'conceptId': '900000000000012004', 'active':...",[],Australian Digital Health Agency,20220131.0,[32570271000036106],Managed Service,NaN
4,Belgian Edition,SNOMEDCT-BE,MAIN/SNOMEDCT-BE,True,2024-09-08-033238,be,"{'shortName': 'SNOMEDCT-BE', 'importDate': '20...","{'en': 'English', 'nl': 'Dutch', 'fr': 'French...","[{'conceptId': '900000000000012004', 'active':...",[],"Belgian Federal Public Service Health, Food Ch...",20240901.0,"[900000000000509007, 21000172104, 31000172101]",Managed Service,en


In [13]:
snomed_server.get_snomed_versions('SNOMEDCT')

,shortName,importDate,parentBranchPath,effectiveDate,version,description,branchPath
0,SNOMEDCT,2019-07-30T21:15:05.887Z,MAIN,20020131,2002-01-31,SNOMEDCT 20020131 import.,MAIN/2002-01-31
1,SNOMEDCT,2019-07-30T21:20:23.200Z,MAIN,20020731,2002-07-31,SNOMEDCT 20020731 import.,MAIN/2002-07-31
2,SNOMEDCT,2019-07-30T21:25:16.053Z,MAIN,20030131,2003-01-31,SNOMEDCT 20030131 import.,MAIN/2003-01-31
3,SNOMEDCT,2019-07-30T21:28:57.204Z,MAIN,20030731,2003-07-31,SNOMEDCT 20030731 import.,MAIN/2003-07-31
4,SNOMEDCT,2019-07-30T21:31:37.482Z,MAIN,20040131,2004-01-31,SNOMEDCT 20040131 import.,MAIN/2004-01-31
...,...,...,...,...,...,...,...
68,SNOMEDCT,2024-04-10T13:58:10.666Z,MAIN,20240501,2024-05-01,SNOMEDCT 20240501 import.,MAIN/2024-05-01
69,SNOMEDCT,2024-05-15T14:11:34.062Z,MAIN,20240601,2024-06-01,SNOMEDCT 20240601 import.,MAIN/2024-06-01
70,SNOMEDCT,2024-06-12T13:17:52.291Z,MAIN,20240701,2024-07-01,SNOMEDCT 20240701 import.,MAIN/2024-07-01
71,SNOMEDCT,2024-07-17T13:23:22.598Z,MAIN,20240801,2024-08-01,SNOMEDCT 20240801 import.,MAIN/2024-08-01


In [18]:
snomed_server.set_branch('MAIN/2024-09-01')